# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('churn').getOrCreate()

In [3]:
spark

### Check out the data

In [4]:
data = spark.read.csv('./data/customer_churn.csv',inferSchema=True, header = True )

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

En PySpark, el VectorAssembler es una herramienta útil que se utiliza para combinar múltiples columnas de un DataFrame en una sola columna de tipo vector. Esto es útil cuando estás trabajando con algoritmos de aprendizaje automático en Spark, ya que muchos de estos algoritmos requieren que las características de entrada estén en un único vector.

### Format for MLlib

In [10]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol = 'features')

In [13]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features','churn')

### Test Train Split

In [16]:
train_churn,test_churn =final_data.randomSplit([0.7,0.3])

### Fit the model

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:

lr_churn = LogisticRegression(labelCol='churn')

In [19]:
fit_churn = lr_churn.fit(train_churn)

In [20]:
training_sum = fit_churn.summary

In [22]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                590|                590|
|   mean|0.16101694915254236|0.12033898305084746|
| stddev| 0.3678584604312919|0.32563359215308657|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
pred_and_labels = fit_churn.evaluate(test_churn)

In [26]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.25262776483266...|[0.99479350223692...|       0.0|
|[26.0,8787.39,1.0...|    1|[1.13558689715947...|[0.75686846688892...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.80555555106729...|[0.99699825595046...|       0.0|
|[29.0,11274.46,1....|    0|[4.87891107765180...|[0.99245211278772...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.89756302215209...|[0.98011224767117...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.42470637562271...|[0.98816404016701...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.63820737542633...|[0.97437448996820...|       0.0|
|[30.0,12788.37,0....|    0|[3.09006267120459...|[0.95648097381580...|       0.0|
|[31.0,11297.57,1....|    1|[1.35914227368358...|[0.79562025935172...|       0.0|
|[32.0,5756.12,0

### Using AUC

In [27]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',labelCol='churn')

In [28]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [29]:
auc

0.7693404634581106

### Predict on new data

In [30]:
final_lr_model = lr_churn.fit(final_data)

In [32]:
new_customers = spark.read.csv('./data/new_customers.csv',inferSchema = True, header=True)

In [33]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [34]:
test_new_customers = assembler.transform(new_customers)

In [35]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [38]:
results = final_lr_model.transform(test_new_customers)

In [40]:
results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [41]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                  6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|               null|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|         